In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
df = pd.read_csv(r"E:\Dataset\ICRISAT-District Level Data.csv")
df.head()

,Dist Code,Year,State Code,State Name,Dist Name,RICE AREA (1000 ha),RICE YIELD (Kg per ha)
0,1,1990,14,Chhattisgarh,Durg,397.9,1210.0
1,1,1991,14,Chhattisgarh,Durg,393.2,1293.0
2,1,1992,14,Chhattisgarh,Durg,398.4,1291.0
3,1,1993,14,Chhattisgarh,Durg,410.2,1387.0
4,1,1994,14,Chhattisgarh,Durg,430.1,1399.0


In [3]:
unwanted = ["Dist Code", "State Code"]
df = df.drop(unwanted, axis=1)

In [16]:
X = df.iloc[:, :4]
print(X.head(67))
y = df.iloc[:, -1].values
y.reshape(-1, 1)

    Year    State Name Dist Name  RICE AREA (1000 ha)
0   1990  Chhattisgarh      Durg               397.90
1   1991  Chhattisgarh      Durg               393.20
2   1992  Chhattisgarh      Durg               398.40
3   1993  Chhattisgarh      Durg               410.20
4   1994  Chhattisgarh      Durg               430.10
..   ...           ...       ...                  ...
62  1996  Chhattisgarh    Raipur               860.40
63  1997  Chhattisgarh    Raipur               918.60
64  1998  Chhattisgarh    Raipur               513.70
65  1999  Chhattisgarh    Raipur               537.61
66  2000  Chhattisgarh    Raipur               513.07

[67 rows x 4 columns]


array([[1210.  ],
       [1293.  ],
       [1291.  ],
       ...,
       [3388.63],
       [3338.59],
       [2653.18]])

In [5]:
le = LabelEncoder()
ct = ColumnTransformer(transformers=[
    ("encoder", OneHotEncoder(sparse_output=False), ["State Name", "Dist Name"]),
    ("scaler", StandardScaler(), ["Year", "RICE AREA (1000 ha)"])
    ], remainder="passthrough")
X = ct.fit_transform(X)

In [6]:
sc = StandardScaler()
y = sc.fit_transform(y.reshape(-1, 1))

In [7]:
X

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -1.80554472,  3.17575153],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -1.67985215,  3.12821813],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -1.55415958,  3.18080828],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.58815463, -0.49645666],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.46246206, -0.36518357],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.58815463, -0.31239115]])

In [8]:
y

array([[-0.59221072],
       [-0.51446624],
       [-0.5163396 ],
       ...,
       [ 1.44846964],
       [ 1.40159814],
       [ 0.75958792]])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestRegressor
rc = RandomForestRegressor(n_estimators=100)
rc.fit(X_train, y_train)

C:\Users\tharu\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

y_pred = rc.predict(X_test)

print(f"Mean Absolute Error : {mean_absolute_error(y_test, y_pred)}")
print(f"Mean Squared Error : {mean_squared_error(y_test, y_pred)}")
print(f"Root Mean Squared Error : {np.sqrt(mse)}")
print(f"R² Score: {r2_score(y_test, y_pred)}")

Mean Absolute Error : 0.21824632601856644
Mean Squared Error : 0.10688482816156644
Root Mean Squared Error : 0.32693245198598203
R² Score: 0.8923748829902772


In [41]:
input_data = pd.DataFrame([[2000, "Madhya Pradesh", "Chhatarpur", 18.7]], 
                          columns=['Year', 'State Name', 'Dist Name', 'RICE AREA (1000 ha)'])
vs = ct.transform(input_data)
lio =  [rc.predict(vs)]
sc.inverse_transform(lio)

array([[448.98]])

In [22]:
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor(n_estimators=100)
parameters = [{"criterion":["squared_error", "absolute_error", "friedman_mse"], "max_features": ["sqrt", "log2"]}]

In [33]:
print(ct.feature_names_in_)

['Year' 'State Name' 'Dist Name' 'RICE AREA (1000 ha)']


In [23]:
gs = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='neg_mean_squared_error',
    cv=10
)

In [ ]:
gs.fit(X_train, y_train)

C:\Users\tharu\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\tharu\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\tharu\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\tharu\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_met

In [ ]:
print(f"Best accuracy is {gs.best_score_*100} \nBest parameter is {gs.best_params_}")